# Effective Bounds: A<sup>eff</sup>, B<sup>eff</sup> etc.

Implementation of $A^{eff}$, $B^{eff}$ using the technique illustrated in **Effective Bounds: A', B'**. As before, the forumlae below are cut and pasted into markdown cells of this document from LaTeX provided by MathJax on the [web page](http://michaelnielsen.org/polymath1/index.php?title=Polymath15_test_problem).

$$A^{eff} := \frac{1}{8} \exp( \frac{t}{4} \alpha_1(\frac{1-y+ix}{2})^2 ) H_{0,1}(\frac{1-y+ix}{2}) \sum_{n=1}^N \frac{1}{n^{\frac{1-y+ix}{2} + \frac{t \alpha_1(\frac{1-y+ix}{2})}{2} - \frac{t}{4} \log n}}$$

$$B^{eff} := \frac{1}{8} \exp( \frac{t}{4} \overline{\alpha_1(\frac{1+y+ix}{2})}^2 ) \overline{H_{0,1}(\frac{1+y+ix}{2})} \sum_{n=1}^N \frac{1}{n^{\frac{1+y-ix}{2} + \frac{t \overline{\alpha_1(\frac{1+y+ix}{2})}}{2} - \frac{t}{4} \log n}}$$

(Note: The following probably should read $B_0^{eff}$ rather than $B^{eff}$ as it is just the previous formula with N=1.)

$$B^{eff} := \frac{1}{8} \exp( \frac{t}{4} \overline{\alpha_1(\frac{1+y+ix}{2})}^2 ) \overline{H_{0,1}(\frac{1+y+ix}{2})}$$

$$H_{0,1}(s) := \frac{s (s-1)}{2} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s}{2} - \frac{1}{2}) \log \frac{s}{2} - \frac{s}{2} )$$

$$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi}$$

$$N := \lfloor \sqrt{ \frac{T'}{2\pi}} \rfloor$$

$$T' := \frac{x}{2} - \frac{\pi t}{8}.$$

Recall the convention that if $z= x+iy$, then $s=\frac{1+zi}{2}=\frac{1-y+ix}{2}$. It will also be helpful to note that $\frac{1+y-ix}{2}= 1-s$, and that $\frac{1+y+ix}{2} = \overline{1-s}$. Using these relationships, the formulae for $A^{eff}$ and $B^{eff}$ may be written as,

$$A^{eff} := \frac{1}{8} \exp( \frac{t}{4} \alpha_1(s)^2 ) H_{0,1}(s) \sum_{n=1}^N \frac{1}{n^{s + \frac{t \alpha_1(s)}{2} - \frac{t}{4} \log n}}$$

$$B^{eff} := \frac{1}{8} \exp( \frac{t}{4} \overline{\alpha_1(\overline{1-s})}^2 ) \overline{H_{0,1}(\overline{1-s}}) \sum_{n=1}^N \frac{1}{n^{1-s + \frac{t \overline{\alpha_1(\overline{1-s})}}{2} - \frac{t}{4} \log n}}$$

(Note that both $\frac{1+y+ix}{2}=1-s$ and $\frac{1+y-ix}=\overline{1-s}$ occur in the formula for $B^{eff},$ while only $s$ appears in the expression for $A^{eff}$ This "asymmetry" does *not* appear to be a typo, since it is consistent with the Python implemetation, [Ht_Effective(z, t)](https://github.com/km-git-acc/dbn_upper_bound/blob/d1cc87111711f7d1e826b9d253c1453a01489561/dbn_upper_bound/python/mputility.py#L386), specifically with the definition of `s1` as $\frac{1-y+ix}{2} = s$, the use of `1-s1` or $\frac{1+y-ix}{2}=1-s$ and in the associated summation loop, and the use of `s2` or $\overline{1-s}$ when calculating `alph2`.)

The formulae for $T'$ and $N$ may be combined as follows. Since $s = \frac{1-y+ix}{2}$, we have $\frac{x}{2}= imag(s)$, hence $T' = imag(s)- \frac{\pi t}{8}$ so that $N = \lfloor \sqrt{ \frac{imag(s)- \frac{\pi t}{8}}{2\pi}} \rfloor$

In [14]:
using DBNUpperBound
using DBNUpperBound.Asymptotics

Nseff(s,t) = floor(Int, √((imag(s) - π*t/8)/(2*π))) # Admissable function definition syntax in Julia

Nseff (generic function with 1 method)

Implementing $\alpha_1$ as $\alpha$eff:

$$\alpha_1(s) := \frac{1}{2s} + \frac{1}{s-1} + \frac{1}{2} \log \frac{s}{2\pi}$$

In [15]:
αeff(s) = 1/(2*s) + 1/(s-1) + 1/2*log(s/(2*π))

αeff (generic function with 1 method)

Continuing with $H_{0,1}$

$$H_{0,1}(s) := \frac{s (s-1)}{2} \pi^{-s/2} \sqrt{2\pi} \exp( (\frac{s}{2} - \frac{1}{2}) \log \frac{s}{2} - \frac{s}{2} )$$

In [16]:
H01(s) = (s*(s-1)/2)*√(2*π)*bigexp(-s/2*log(π) + (s/2-1/2)*log(s/2) - s/2)

H01 (generic function with 1 method)

The denominator of the $A^{eff}$ summand is,

$$ n^{\frac{1-y+ix}{2} + \frac{t \alpha_1(\frac{1-y+ix}{2})}{2} - \frac{t}{4} \log n}$$

The summand itself may thus be implemented with a negative exponent as:

In [20]:
function aeff(s::T1, t::T2, n::Int) where {T1<:Number, T2<:Real}
    return bigexp(-log(n)*(s + t*αeff(s)/2 - t/4*log(n)))
end

aeff (generic function with 1 method)

The denominator of the $B^{eff}$ summand is,

$$n^{1-s + \frac{t \overline{\alpha_1(\overline{1-s})}}{2} - \frac{t}{4} \log n}$$

and the summand itself may be implemented as:


In [22]:
function beff(s::T1, t::T2, n::Int) where {T1<:Number, T2<:Real}
    return bigexp(-log(n)*( 1-s + t*αeff((1-s)')'/2 - t/4*log(n)))
end

beff (generic function with 1 method)

The multiplier of the sum in $A^{eff}$ is

$$ \frac{1}{8} \exp( \frac{t}{4} \alpha_1(s)^2 ) H_{0,1}(s)$$

which may be implemented as

In [19]:
function Aeff_μ(s::T1, t::T2) where {T1<:Number, T2<:Real}
    return 1/8*bigexp(t/4*αeff(s)^2)*H01(s)
end

Aeff_μ (generic function with 1 method)

The multiplier of the sum in $B^{eff}$ is

$$\frac{1}{8} \exp( \frac{t}{4} \overline{\alpha_1(\overline{1-s})}^2 ) \overline{H_{0,1}(\overline{1-s}})$$

which may be implemented as,

In [23]:
function Beff_μ(s::T1, t::T2) where {T1 <: Number, T2 <: Real}
    return 1/8*bigexp(t/4*(αeff((1-s)')')^2)*H01((1-s)')
end

Beff_μ (generic function with 1 method)